In [1]:
import geopandas as gpd
import rasterio

In [2]:
pointdata = gpd.read_file(r"C:\Users\likith\Desktop\QGIS\project\landsat\CPCB_sensors_Sept\reproj_CPCB.shp")

In [3]:
print(pointdata.crs)

epsg:32643


In [4]:
pointdata.columns

Index(['OID_', 'Name', 'FolderPath', 'SymbolID', 'AltMode', 'Base', 'Snippet',
       'PopupInfo', 'HasLabel', 'LabelID', 'geometry'],
      dtype='object')

In [5]:
pointdata = pointdata[[ 'Name','geometry']]

In [6]:
pointdata.head()

,Name,geometry
0,Silk board,POINT Z (784601.854 1429448.237 0.000)
1,"BWSSD, Kadabesnahalli",POINT Z (790966.423 1431424.929 0.000)
2,BTM layout Bangalore,POINT Z (781590.905 1429038.664 0.000)
3,Jayanagar 5th block,POINT Z (780484.618 1429805.506 0.000)
4,Hombegowda nagar,POINT Z (781032.712 1431768.080 0.000)


In [7]:
print(pointdata['geometry'])

0    POINT Z (784601.854 1429448.237 0.000)
1    POINT Z (790966.423 1431424.929 0.000)
2    POINT Z (781590.905 1429038.664 0.000)
3    POINT Z (780484.618 1429805.506 0.000)
4    POINT Z (781032.712 1431768.080 0.000)
5    POINT Z (775538.512 1433193.368 0.000)
6    POINT Z (778382.490 1435848.048 0.000)
7    POINT Z (775875.579 1437428.579 0.000)
8    POINT Z (780472.693 1441789.649 0.000)
9    POINT Z (770506.005 1441452.247 0.000)
Name: geometry, dtype: geometry


In [8]:
import os
files = os.listdir("./purple_air/u_Data")

In [9]:
files = files[-3:]

In [21]:
files[-2]

'2021_04_03.csv'

In [22]:
import pandas as pd
data = pd.read_csv("./purple_air/u_Data/"+str(files[-2]))

In [13]:
data=data.drop(['Unnamed: 9','Unnamed: 0'],axis = 1)

In [14]:
'2021-04-19 10:00:00 UTC'

'2021-04-19 10:00:00 UTC'

In [129]:
data['created_at'][0]

'2021-03-02 00:00:00 UTC'

In [130]:
df = pd.DataFrame()
for i in range(len(data['created_at'])):
    if data['created_at'][i] == '2021-03-02 10:00:00 UTC':
        df = df.append(data.iloc[i])

In [131]:
len(df)

34

In [132]:
new_df = pd.merge(df, pointdata,  how='left', left_on=['Lat','Long'], right_on = ['Lat','Long'])

In [133]:
new_df=new_df.dropna()

In [134]:
len(new_df)

28

In [91]:
raster = rasterio.open(r"C:\Users\likith\Desktop\QGIS\project\landsat\preprocessing_and _data_extraction\image\2021_04_19\L1\LC08_L1TP_144051_20210419_20210424_02_T1_B5.tif")

In [92]:
raster_l2 = rasterio.open(r"C:\Users\likith\Desktop\QGIS\project\landsat\preprocessing_and _data_extraction\image\2021_04_19\L2\LC08_L2SP_144051_20210419_20210424_02_T1_SR_B5.tif")

In [93]:
ex = []
val = []
for i in pointdata['geometry']:
    x = i.xy[0][0]
    y = i.xy[1][0]
    row,col = raster.index(x,y)
    ex.append(raster.read(1)[row,col])
    row,col = raster_l2.index(x,y)
    val.append(raster_l2.read(1)[row,col])
l1 = []
for i in ex:
    l1.append(((i*0.00002)-0.100000)/0.82544361547)
l2 =[]
for i in val:
    l2.append((i*0.0000275)-0.2)
#correct

In [94]:
ar = []
for i in range(len(l2)):
    ar.append(l1[i]-l2[i])

In [95]:
pointdata['AR_B5']=ar

In [96]:
pointdata.to_csv("2021-04-19_cpcb.csv")

In [44]:
complete_data=complete_data.append(new_df)

NameError: name 'complete_data' is not defined

In [157]:
complete_data.to_csv("purp_data.csv")

In [87]:
new_df.head()

,created_at,PM1.0_CF1_ug/m3,PM2.5_CF1_ug/m3,PM10.0_CF1_ug/m3,UptimeMinutes,RSSI_dbm,Temperature_F,Humidity_%,PM2.5_ATM_ug/m3,Lat,Long,Object_ID,geometry,AR_B1,AR_B2,AR_B3,AR_B4,AR_B5
0,2021-04-19 10:00:00 UTC,38.12,54.57,55.78,6297.0,-28.43,99.13,23.67,43.25,12.997507,77.605669,1.0,POINT (782653.291 1438306.293),0.111770,0.080799,0.032913,0.021257,0.006816
2,2021-04-19 10:00:00 UTC,40.13,53.98,56.53,55294.0,-31.77,111.00,11.83,42.99,13.045313,77.573394,5.0,POINT (779096.488 1443562.179),0.116619,0.085388,0.034818,0.022671,0.009714
3,2021-04-19 10:00:00 UTC,34.98,49.97,56.74,55294.0,-36.40,111.93,12.83,41.07,13.045878,77.574580,2.0,POINT (779224.559 1443626.024),0.116210,0.083428,0.031805,0.020028,0.008862
4,2021-04-19 10:00:00 UTC,27.18,39.20,42.27,55295.0,-43.87,112.40,12.13,35.23,13.045371,77.574708,4.0,POINT (779239.019 1443570.046),0.115493,0.083824,0.032182,0.022341,0.008709
5,2021-04-19 10:00:00 UTC,36.41,51.77,55.93,1025.0,-62.27,96.10,24.27,41.96,13.054695,77.542158,6.0,POINT (775696.365 1444566.447),0.118852,0.089295,0.040089,0.030143,0.009034


In [88]:
complete_data = pd.DataFrame()

In [31]:
data.head()

,created_at,PM1.0_CF1_ug/m3,PM2.5_CF1_ug/m3,PM10.0_CF1_ug/m3,UptimeMinutes,RSSI_dbm,Temperature_F,Humidity_%,PM2.5_ATM_ug/m3,Lat,Long
0,2021-04-19 00:00:00 UTC,34.04,54.09,55.89,5697.0,-24.90,86.43,49.30,43.02,12.997507,77.605669
1,2021-04-19 01:00:00 UTC,31.93,49.81,51.35,5757.0,-26.30,86.43,48.73,40.98,12.997507,77.605669
2,2021-04-19 02:00:00 UTC,34.78,54.05,55.86,5817.0,-27.03,86.87,48.13,42.99,12.997507,77.605669
3,2021-04-19 03:00:00 UTC,34.83,53.89,55.61,5877.0,-28.47,88.47,45.57,42.95,12.997507,77.605669
4,2021-04-19 04:00:00 UTC,34.85,53.08,54.58,5937.0,-27.90,90.83,41.17,42.60,12.997507,77.605669


In [6]:
raster = rasterio.open(r"C:\Users\likith\Desktop\QGIS\project\landsat\preprocessing_and _data_extraction\01_06_2019\pre_processing\Data\B1.tif")

In [7]:
ex = []
for i in pointdata['geometry']:
    x = i.xy[0][0]
    y = i.xy[1][0]
    row,col = raster.index(x,y)
    ex.append(raster.read(1)[row,col])
#correct

In [ ]:
val = []
for i in l1:
    val.append(((i*0.00002)-0.100000)/0.90618548032)

In [8]:
ex

[0.10640396,
 0.12757063,
 0.121043354,
 0.12495566,
 0.1270487,
 0.11452732,
 0.1208031,
 0.11487005,
 0.11490812,
 0.12607771,
 0.0971471,
 0.10723242,
 0.12174301,
 0.105604656,
 0.13545193,
 0.11903602,
 0.11562781,
 0.13162771,
 0.13877492,
 0.11413008,
 0.11103977,
 0.11521808,
 0.11608982,
 0.119799644,
 0.11300772,
 0.11607061,
 0.11943128,
 0.10812491,
 0.13450122,
 0.123935014,
 0.12150025,
 0.12042467,
 0.120356925,
 0.123340376,
 0.12643814,
 0.104733564,
 0.15491647]

In [69]:
raster_toa = rasterio.open(r"C:\Users\likith\Desktop\QGIS\project\landsat\preprocessing_and _data_extraction\01_06_2019\pre_processing\TOA\01_06_TOA (1).tif")

In [9]:
raster_l1 = rasterio.open(r"C:\Users\likith\Desktop\QGIS\project\landsat\preprocessing_and _data_extraction\01_06_2019\L1C2\LC08_L1TP_144051_20190601_20200828_02_T1_B1.tif")

In [10]:
l1 = []
for i in pointdata['geometry']:
    x = i.xy[0][0]
    y = i.xy[1][0]
    row,col = raster_l1.index(x,y)
    l1.append(raster_l1.read(1)[row,col])
l1

[12282,
 13015,
 13017,
 12948,
 12863,
 11886,
 12768,
 13436,
 12095,
 13056,
 13073,
 11685,
 12812,
 12378,
 12376,
 11650,
 11509,
 12742,
 11640,
 11821,
 12065,
 12476,
 11527,
 12692,
 12585,
 12611,
 13133,
 12515,
 12953,
 12628,
 12361,
 12266,
 12533,
 12564,
 12656,
 13490,
 12484]

In [15]:
val = []
for i in l1:
    val.append(((i*0.00002)-0.100000)/0.90618548032)

In [16]:
import statistics
statistics.variance(val)

0.00013116358511757403

In [17]:
raster_l2 = rasterio.open(r"C:\Users\likith\Desktop\QGIS\project\landsat\preprocessing_and _data_extraction\01_06_2019\L2C2\LC08_L2SP_144051_20190601_20200828_02_T1_SR_B1.tif")

In [25]:
l2 = []
for i in pointdata['geometry']:
    x = i.xy[0][0]
    y = i.xy[1][0]
    row,col = raster_l2.index(x,y)
    l2.append(raster_l2.read(1)[row,col])

In [26]:
ar = []
for i in l2:
    ar.append((i*0.0000275)-0.2)

In [29]:
ar

[0.058252499999999985,
 0.053659999999999985,
 0.06023250000000002,
 0.054759999999999975,
 0.050744999999999985,
 0.04117499999999999,
 0.054842500000000016,
 0.07588,
 0.045520000000000005,
 0.05608000000000002,
 0.085395,
 0.04392499999999999,
 0.05489749999999999,
 0.06122250000000001,
 0.03133,
 0.03133,
 0.031549999999999995,
 0.043429999999999996,
 0.011364999999999986,
 0.040102499999999985,
 0.04871,
 0.05382500000000001,
 0.031494999999999995,
 0.054127499999999995,
 0.058499999999999996,
 0.05602499999999999,
 0.06446750000000001,
 0.06180000000000002,
 0.04532749999999999,
 0.048545000000000005,
 0.044942499999999996,
 0.04386999999999999,
 0.04997499999999999,
 0.0476925,
 0.046674999999999994,
 0.08723750000000002,
 0.0143075]

In [27]:
ls = []
for i in range(len(val)):
    ls.append(val[i] - ar[i])

In [28]:
ls

[0.10246514904969609,
 0.1232353525313532,
 0.11670699360497297,
 0.12065662656508988,
 0.12279563093624835,
 0.11080271647297543,
 0.11660142993930775,
 0.11030704852832132,
 0.11107045866624465,
 0.12172024454055907,
 0.09278044366632743,
 0.1036165385741854,
 0.11751753355894337,
 0.10161392058344651,
 0.13146227950982672,
 0.11543906978583887,
 0.11210712409564291,
 0.1274400959822503,
 0.13518336441773987,
 0.1104406315803319,
 0.10721834256194762,
 0.11117433319081677,
 0.11255939375822113,
 0.11563906914175529,
 0.10890502170309596,
 0.11195385566015334,
 0.11503217587492143,
 0.10406008412640289,
 0.13019947924913935,
 0.11980905478592163,
 0.11751872145767822,
 0.11649452046073763,
 0.11628235378898108,
 0.119249040430088,
 0.12229702981659887,
 0.10014135751605596,
 0.15086839748529596]

In [68]:
(((raster_l1.read(1)*0.00002) - 0.100000)/0.90618548032) - ((raster_l2.read(1)*0.0000275)-0.2)

array([[0.08964732, 0.08964732, 0.08964732, ..., 0.08964732, 0.08964732,
        0.08964732],
       [0.08964732, 0.08964732, 0.08964732, ..., 0.08964732, 0.08964732,
        0.08964732],
       [0.08964732, 0.08964732, 0.08964732, ..., 0.08964732, 0.08964732,
        0.08964732],
       ...,
       [0.08964732, 0.08964732, 0.08964732, ..., 0.08964732, 0.08964732,
        0.08964732],
       [0.08964732, 0.08964732, 0.08964732, ..., 0.08964732, 0.08964732,
        0.08964732],
       [0.08964732, 0.08964732, 0.08964732, ..., 0.08964732, 0.08964732,
        0.08964732]])

In [67]:
raster.read(1)

array([[-3.4028235e+38, -3.4028235e+38, -3.4028235e+38, ...,
        -3.4028235e+38, -3.4028235e+38, -3.4028235e+38],
       [-3.4028235e+38, -3.4028235e+38, -3.4028235e+38, ...,
        -3.4028235e+38, -3.4028235e+38, -3.4028235e+38],
       [-3.4028235e+38, -3.4028235e+38, -3.4028235e+38, ...,
        -3.4028235e+38, -3.4028235e+38, -3.4028235e+38],
       ...,
       [-3.4028235e+38, -3.4028235e+38, -3.4028235e+38, ...,
        -3.4028235e+38, -3.4028235e+38, -3.4028235e+38],
       [-3.4028235e+38, -3.4028235e+38, -3.4028235e+38, ...,
        -3.4028235e+38, -3.4028235e+38, -3.4028235e+38],
       [-3.4028235e+38, -3.4028235e+38, -3.4028235e+38, ...,
        -3.4028235e+38, -3.4028235e+38, -3.4028235e+38]], dtype=float32)

In [73]:
l2 = []
for i in pointdata['geometry']:
    x = i.xy[0][0]
    y = i.xy[1][0]
    row,col = raster_l2.index(x,y)
    l2.append(((((raster_l1.read(1)*0.00002) - 0.100000)/0.90618548032) - ((raster_l2.read(1)*0.0000275)-0.2))[row,col])

In [62]:
ls = []
for i in range(len(l1)):
    ls.append(l1[i] - l2[i])
ex

[0.10640396,
 0.12757063,
 0.121043354,
 0.12495566,
 0.1270487,
 0.11452732,
 0.1208031,
 0.11487005,
 0.11490812,
 0.12607771,
 0.0971471,
 0.10723242,
 0.12174301,
 0.105604656,
 0.13545193,
 0.11903602,
 0.11562781,
 0.13162771,
 0.13877492,
 0.11413008,
 0.11103977,
 0.11521808,
 0.11608982,
 0.119799644,
 0.11300772,
 0.11607061,
 0.11943128,
 0.10812491,
 0.13450122,
 0.123935014,
 0.12150025,
 0.12042467,
 0.120356925,
 0.123340376,
 0.12643814,
 0.104733564,
 0.15491647]

In [77]:
for i in range(len(l2)):
    print((-l2[i]+ex[i])*100/ex[i])

3.7017538277573387
3.3983344116349508
3.5824855432334584
3.440448375773635
3.347590393738529
3.2521548591799805
3.47811671426324
3.9723152975328677
3.33976642296798
3.4561755499379805
4.494890760594252
3.372005413297961
3.470815275300942
3.778938928303635
2.9454348392336844
3.0217316551561453
3.044843842002389
3.1814066608231246
2.588041269798271
3.232669401543791
3.441496661320437
3.5096465494422238
3.0411168501996384
3.4729440612644513
3.6304557302541913
3.546769926582472
3.683376532086866
3.759381248606972
3.19829038364974
3.3291312035827243
3.276974408845685
3.263577474495229
3.385406405637329
3.3171094764672047
3.2752071677704175
4.384655949752693
2.6130649119600573
